In [1]:
import pandas as pd
df = pd.read_csv('../Data/lstm.csv')
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


In [2]:
df.category.unique()

array(['food', 'sports'], dtype=object)

> 중복된것을 제거하고 대소문자를 찢어놔야된다.


In [3]:
# 데이터에 사용된 중복 없는 전체 단어 갯수를 파악
results = set() # 중복 제거 : set()
df['paragraph'].str.lower().str.split().apply(results.update)
vocab_size = len(results)
vocab_size


536

In [4]:
# 단어를 숫자로 인코딩
paragraphs = df['paragraph'].to_list()
paragraphs[0:5]

['dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait',
 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations',
 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table',
 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition',
 'the biscuits and gravy was too salty two people in my group had the gravy and all thoug

In [5]:
from tensorflow import keras
# keras에서 setseed한 것
keras.utils.set_random_seed(1)

In [6]:
# One hot encoding
encoded_paragraphs = [keras.preprocessing.text.one_hot(paragraph, vocab_size) for paragraph in paragraphs]
encoded_paragraphs

[[188,
  49,
  158,
  309,
  265,
  78,
  63,
  49,
  427,
  420,
  427,
  123,
  354,
  295,
  6,
  503,
  266,
  145,
  351,
  262,
  267,
  146,
  83,
  21,
  461,
  417,
  135,
  506,
  145,
  452,
  241,
  3,
  88,
  84,
  192,
  393,
  401,
  521,
  155,
  181,
  308,
  135,
  213,
  523,
  14,
  37,
  97,
  520,
  8,
  427,
  411,
  145,
  388],
 [177,
  6,
  503,
  35,
  262,
  181,
  83,
  354,
  16,
  283,
  470,
  407,
  417,
  419,
  388,
  532,
  344,
  35,
  143],
 [515,
  298,
  142,
  241,
  103,
  271,
  417,
  190,
  237,
  437,
  298,
  155,
  341,
  190,
  103,
  271,
  49,
  331,
  205,
  404,
  270,
  325,
  258,
  238,
  245,
  344,
  241,
  145,
  15,
  435,
  520,
  298,
  378,
  417,
  228,
  30,
  145,
  388,
  205,
  374,
  3,
  192,
  293],
 [355,
  185,
  259,
  487,
  535,
  427,
  483,
  417,
  285,
  464,
  417,
  483,
  25,
  427,
  16,
  521,
  235,
  417,
  427,
  25,
  521,
  341,
  136,
  378,
  359,
  447,
  427,
  25,
  50,
  8,
  19,
  242,
  35

In [7]:
# 데이터에서 가장 긴 문장의 단어 갯수 확인
max_length = 0
for row in df['paragraph']:
    if len(row.split(" ")) > max_length:
        max_length = len(row.split(" "))
print(max_length)    

91


In [8]:
# 문장마다 단어 갯수가 다르므로 sequencce padding을 넣어서 문장의 길이를 동일하게 만듭니다.
padded_paragraphs_encoding = keras.preprocessing.sequence.pad_sequences(
    encoded_paragraphs, maxlen = max_length, padding='post'
)
padded_paragraphs_encoding


array([[188,  49, 158, ...,   0,   0,   0],
       [177,   6, 503, ...,   0,   0,   0],
       [515, 298, 142, ...,   0,   0,   0],
       ...,
       [ 98,   2,   3, ...,   0,   0,   0],
       [354, 427,  66, ...,   0,   0,   0],
       [219, 506, 427, ...,   0,   0,   0]], dtype=int32)

In [9]:
# 분류항목(food, sports)를 수치로 변경하기

categories = df['category'].to_list()

def category_encode(category):
    if category == "food":
        return [1,0]
    else:
        return [0,1]

In [10]:
encoded_category = [category_encode(category) for category in categories]
encoded_category

[[1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1]]

In [11]:
# Feature 확인
print(encoded_paragraphs[0])

[188, 49, 158, 309, 265, 78, 63, 49, 427, 420, 427, 123, 354, 295, 6, 503, 266, 145, 351, 262, 267, 146, 83, 21, 461, 417, 135, 506, 145, 452, 241, 3, 88, 84, 192, 393, 401, 521, 155, 181, 308, 135, 213, 523, 14, 37, 97, 520, 8, 427, 411, 145, 388]


In [12]:
len(encoded_paragraphs[19])

73

---
### 주제를 분류하는 모델 구현하기

In [13]:
model = keras.Sequential()
# 문맥 생성 단계
model.add(keras.layers.Embedding(vocab_size, 5, input_length=max_length))
# embedding 레이어는 인덱스를 받아 5차원 벡터의 embedding을 출력합니다. 
# 5차원인 이유는 과거, 과거진행, 과거완료, 현재, 현재진행, 현재완료 ? 아님 5형식?
model.add(keras.layers.LSTM(64))
# 분류단계
model.add(keras.layers.Dense(32, activation='relu')) # LSTM 반값
model.add(keras.layers.Dense(2, activation='softmax')) # 문장은 확률값을 구해야하므로

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')

In [15]:
import numpy as np
train_X = np.array(padded_paragraphs_encoding)
train_Y = np.array(encoded_category)

In [16]:
# 데이터가 많으면 batch_size를 써줘야된다.
model.fit(train_X, train_Y, batch_size=10, epochs=50)

Epoch 1/50


2022-07-22 14:34:42.186116: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2/2 [==============================] - 1s 10ms/step - loss: 0.6941 - accuracy: 0.5000
Epoch 2/50
2/2 [==============================] - 0s 10ms/step - loss: 0.6926 - accuracy: 0.5500
Epoch 3/50
2/2 [==============================] - 0s 11ms/step - loss: 0.6925 - accuracy: 0.5500
Epoch 4/50
2/2 [==============================] - 0s 11ms/step - loss: 0.6923 - accuracy: 0.6000
Epoch 5/50
2/2 [==============================] - 0s 11ms/step - loss: 0.6920 - accuracy: 0.6000
Epoch 6/50
2/2 [==============================] - 0s 10ms/step - loss: 0.6916 - accuracy: 0.6000
Epoch 7/50
2/2 [==============================] - 0s 11ms/step - loss: 0.6911 - accuracy: 0.6000
Epoch 8/50
2/2 [==============================] - 0s 12ms/step - loss: 0.6907 - accuracy: 0.6000
Epoch 9/50
2/2 [==============================] - 0s 11ms/step - loss: 0.6901 - accuracy: 0.6000
Epoch 10/50
2/2 [==============================] - 0s 12ms/step - loss: 0.6897 - accuracy: 0.6000
Epoch 11/50
2/2 [=======================